# Yelp On the Go! 
### Collaborative Filtering Recommender (SVD++)


## Install and Import Library Dependencies  

In [1]:
!pip install surprise

In [2]:
import json
import time
import pandas as pd
from surprise import SVDpp
from surprise import Dataset
from surprise import Reader
from collections import defaultdict
from surprise import accuracy

## Load Preprocessed User, Restaurant, and Review Data 

In [3]:
start = time.time()
with open('MA_users.json', encoding="utf8") as fin:
    print('Reading',fin.name)
    user_data = json.load(fin)
end = time.time()
duration_without_dr = end-start
print("Time taken to load user data: %d seconds" %duration_without_dr)

Reading MA_users.json
Time taken to load user data: 0 seconds


In [4]:
start = time.time()
with open('MA_restaurants.json', encoding="utf8") as fin:
    print('Reading',fin.name)
    restaurant_data = json.load(fin)
end = time.time()
duration_without_dr = end-start
print("Time taken to load restaurant data: %d seconds" %duration_without_dr)

Reading MA_restaurants.json
Time taken to load restaurant data: 0 seconds


In [5]:
start = time.time()
with open('MA_reviews.json', encoding="utf8") as fin:
    print('Reading',fin.name)
    review_data = json.load(fin)
end = time.time()
duration_without_dr = end-start
print("Time taken to load review data: %d seconds" %duration_without_dr)

Reading MA_reviews.json
Time taken to load review data: 3 seconds


In [6]:
print(str(len(user_data)) + ' user records loaded')
print(str(len(review_data)) + ' review records loaded')
print(str(len(restaurant_data)) + ' business records loaded')

13046 user records loaded
465276 review records loaded
6249 business records loaded


## Algorithm Setup
> Create business and user dictionary that will be helpful with viewing the results of the recommender

In [7]:
business_id_dict = dict()
for i,r in enumerate(restaurant_data):
    business_id_dict[r['business_id']] = i

def print_business_info(business_id):
    for attr in restaurant_data[business_id_dict[business_id]].keys():
        print('{0}: {1}'.format(str(attr), str(restaurant_data[business_id_dict[business_id]][attr])))
        
def get_business_attr(business_id, attribute):
    return restaurant_data[business_id_dict[business_id]][attribute]

def get_business_reviews(business_id):
    print('Name:                 User ID:               User Total Review Count         Rating')
    for review in review_data:
        if review['business_id'] == business_id:
            print('{:<15s}{:<15s}{:>20.0f}{:>25.1f}'.format(get_user_attr(review['user_id'], 'name'),review['user_id'],get_user_attr(review['user_id'], 'review_count'), review['stars']))

In [8]:
user_id_dict = dict()
for i,u in enumerate(user_data):
    user_id_dict[u['user_id']] = i

def print_user_info(user_id):
    for attr in user_data[user_id_dict[user_id]].keys():
        print('{0}: {1}'.format(str(attr), str(user_data[user_id_dict[user_id]][attr]) if attr != 'friends' else '[list of user ids]'))

def get_user_attr(user_id, attribute):
    return user_data[user_id_dict[user_id]][attribute]

## Create the Review Dataframe
> Create the rating dictionary by extracting the user_id, business_id, and rating from each review <br>
Import the rating dictionary into a pandas dataframe for fitting the algorithm

In [9]:
rating_dict = {'user_id': [],
               'business_id': [],
               'rating': []}

for review in review_data:
    rating_dict['user_id'].append(review['user_id'])
    rating_dict['business_id'].append(review['business_id'])
    rating_dict['rating'].append(review['stars'])

df = pd.DataFrame(rating_dict)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'business_id', 'rating']], reader)
print('The data frame shape:\t{}'.format(df.shape))

The data frame shape:	(465276, 3)


## Running SVD++ on the Review Dataframe
> Set up the SVD++ hyperparameters found in the downselect process <br>
Run SVD++ on the entire dataset (this takes around 15 minutes)

In [10]:
trainset = data.build_full_trainset()
algo = SVDpp(n_epochs=30, lr_all=0.0025)

# note: this takes quite a while...
start = time.time()
algo.fit(trainset)
end = time.time()
duration_without_dr = end-start
print("Time taken to fit data: %d seconds" %duration_without_dr)

Time taken to fit data: 710 seconds


## Sample User Test and Validation
Select a sample user to view rating predictions and the thie top recommended restaurants

> Get a user_id from the review dataframe

In [24]:
print('User_ID               # of Reviews\n')
print('{}'.format(df['user_id'].value_counts()[:900]))

User_ID               # of Reviews

nl8gWLDo6U6MjqzbBmE_9A    1081
rcU7ysY41qGppbw4pQgjqg     618
ggl6fl-PM5O1WrdReL0l4A     618
1Y0zsJSfWLkfDylH0X1yNQ     588
t903_es-gp3abvdrIQutQA     485
                          ... 
Nw1Qu1V8KcFj8fBA3H2qUw      84
0EFQQB40D24-WuIFH6-rpQ      84
1keSzYrEUFfCNKJjB4MduA      84
J3pXjuQLVOMBFjqsviy2pw      84
-piEModZF7bakZLLW-CXew      84
Name: user_id, Length: 900, dtype: int64


In [25]:
user_id = 'ggl6fl-PM5O1WrdReL0l4A' # sample user

### Sample User Reviews
Get All Reviews Made by the Sample User

In [26]:
df_new_user = df.loc[df['user_id'] == user_id]
print(df_new_user)

                       user_id             business_id  rating
452     ggl6fl-PM5O1WrdReL0l4A  Z6zAW5KYUbOqlv1pa63STw     4.0
1087    ggl6fl-PM5O1WrdReL0l4A  M0fRUDEo9mbnZFXhvshlFw     4.0
2263    ggl6fl-PM5O1WrdReL0l4A  wLpF_Qxw4d48-KR2ltTD_A     4.0
2268    ggl6fl-PM5O1WrdReL0l4A  oz882XuZCxajKo64Opgq_Q     3.0
2796    ggl6fl-PM5O1WrdReL0l4A  yKvlRknp1Feues1rG-aJlg     4.0
...                        ...                     ...     ...
460923  ggl6fl-PM5O1WrdReL0l4A  RNqUyblZzXvg0a2fBjJQQg     4.0
461602  ggl6fl-PM5O1WrdReL0l4A  jIxhHWqxKd5ZWBIEmsFxWQ     5.0
461960  ggl6fl-PM5O1WrdReL0l4A  mlYVUORelx-_gFihRA1s6g     1.0
462326  ggl6fl-PM5O1WrdReL0l4A  y2w6rFaO0XEiG5mFfOsiFA     4.0
464471  ggl6fl-PM5O1WrdReL0l4A  rnwrQjhb7KcV6uoElNTH5w     5.0

[618 rows x 3 columns]


### Make Recommendations to Sample User
Call the Predict Method for Each of the Restaurants the Sample User has Reviewed
> r_ui is the actual review score the sample user gave to the restaurant <br>
est is the estimated review score given by the collaborative filtering algorithm

In [33]:
pred_list = list()
for business_id in business_id_dict:
    if (df_new_user['business_id'] == business_id).any():
        r_ui = df_new_user.loc[df_new_user['business_id'] == business_id, 'rating'].iloc[0]
        pred = algo.predict(user_id, business_id, r_ui = r_ui, verbose=True)
        pred_list.append(pred)

user: ggl6fl-PM5O1WrdReL0l4A item: SI0R_Fw7kFc_2bO_SX2rqw r_ui = 4.00   est = 3.30   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: hZOYhhNMP_yiR7QJT4qawg r_ui = 3.00   est = 4.26   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 96RV4GvZdPahV-brg3hnOQ r_ui = 4.00   est = 4.34   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: kVI7NL59MPnYTx3Lg6PMTw r_ui = 5.00   est = 3.56   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: ecqjHw3jeEApa9eZ622hAA r_ui = 5.00   est = 4.13   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: AG1ETqbneinRRJUeA8PfSA r_ui = 4.00   est = 3.89   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 68p1rT55DCCaB5rIi7Jwng r_ui = 4.00   est = 3.63   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: YMwzbodNWxRxhY1Wz5gE1A r_ui = 5.00   est = 4.00   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: Oj5Seggqo_2FfKSjcAg7yw r_ui = 3.00   est = 3.34   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: DrfWREMxP-0JCt5Pz8_sUQ r_ui = 4.00   est = 3.73   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: gJWbeL6cDgcPvvZSnxo6cQ r_ui = 5.00   est = 3.76   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: tztlzlnaUjMnPEUmcc_ewQ r_ui = 3.00   est = 3.87   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: leBIJOs_IqI_oNDfRZpVFg r_ui = 4.00   est = 3.29   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 6bw34tvKak9ZD0xA7iKM-g r_ui = 5.00   est = 3.33   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: tdIyREwlre3SVU41a4eBgA r_ui = 4.00   est = 2.93   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 7IvP3dv6OXLt7T2_gkNEcg r_ui = 5.00   est = 4.27   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 1x-xpvSGsf6sBhCf045NUA r_ui = 4.00   est = 4.39   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: _RWkck7niE8VJG3zLXK2ig r_ui = 3.00   est = 3.75   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: Hc9l2eLAGTB4lZJI3vRoPw r_ui = 4.00   est = 3.80   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: A--_gxXUMp4caZzQlLPLoA r_ui = 5.00   est = 3.36   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 7uiRFMg1LX7bXdYa5eT3ZA r_ui = 3.00   est = 3.63   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: kLr68qdt6nutZCuCMz5mDg r_ui = 5.00   est = 4.31   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: ltekEILmy2-Y3U8I9Vs5FA r_ui = 4.00   est = 2.94   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: sizgsuBgbH1ShRE5iDTjxA r_ui = 3.00   est = 3.58   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 5NsdznaNSUmH2O0vuo65Pw r_ui = 3.00   est = 4.22   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: cddsQYuoK6qF0BM-r3k-vg r_ui = 2.00   est = 2.89   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: NzvT4LbEqzWoCH2VJ3RMjw r_ui = 2.00   est = 3.04   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: _dT4yoCbm91hA_3YJ1aLeQ r_ui = 3.00   est = 3.09   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: e4r0wRKY_iNdmDffnV0ntw r_ui = 3.00   est = 4.05   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: nVpdObH9RteZ2xILtZIQFw r_ui = 4.00   est = 3.70   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: Xk9yushfKTTPOUeeH-iMQw r_ui = 4.00   est = 4.43   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: WDpPZYEZhf7LLv3PyTeb4Q r_ui = 4.00   est = 3.66   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: CX3sJE-H5nf9bVSnh_khNA r_ui = 5.00   est = 4.15   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: goj3-MeDiCS2soysk6AaSQ r_ui = 3.00   est = 3.60   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 44uFnSanJ1umQRsYFm5zCQ r_ui = 4.00   est = 4.31   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: fvuXHSkSE00DglB2-qHt5w r_ui = 4.00   est = 4.19   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: 4slCpJ_siA2Wfp7-pb3y5Q r_ui = 4.00   est = 3.93   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 09pqlAR4TSjQXNTiB4cXjw r_ui = 4.00   est = 3.91   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: V8_e5MRjfexH4WFBqjDWCg r_ui = 4.00   est = 3.10   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: HG7U-ZdrByIiqpcD25zebQ r_ui = 3.00   est = 3.82   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: THC9oqzIuSOh-dMvoe2N9w r_ui = 4.00   est = 3.71   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: oqdfuREGxgvEe7uQS7xO0A r_ui = 5.00   est = 3.11   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: YF9EI5pIRfFU1RxjNaUIgQ r_ui = 3.00   est = 3.12   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: DfrvJL-6H5i1nsyL6H6VGg r_ui = 5.00   est = 4.24   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: vML0vAWVHQEbFrI3_jS2oA r_ui = 4.00   est = 4.25   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: WsYQF28GdiStUJuLSecIaA r_ui = 5.00   est = 4.38   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 5HtKXpccv60YtqpjszDGBQ r_ui = 5.00   est = 4.40   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: UGs6J02mK_DTxDPfrJoviw r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: ckjbAB-tQ3_QEQmHSytZ6Q r_ui = 3.00   est = 2.87   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 6eWfwUSegYhGvgVemLEKDQ r_ui = 4.00   est = 4.37   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 6dj0t8vx2kdskDMhChLSHQ r_ui = 4.00   est = 3.57   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: A86KZNesn79wjY8_V4zZew r_ui = 5.00   est = 3.71   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: oDoacShrKZFDO-X4NrhKUg r_ui = 4.00   est = 3.85   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: HhuFsOJ5JTF7zkaczqbqcw r_ui = 4.00   est = 3.14   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: 1ajkKqDVs4oyhCSjj7-wqQ r_ui = 5.00   est = 4.62   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: T5_3LoIuOnilA7Z5sdfGJA r_ui = 2.00   est = 3.99   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: j4GcfVzq1BCXGoPtNGOtgg r_ui = 3.00   est = 4.20   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: DmJK-IWY2F0ykgHKxVgnZg r_ui = 3.00   est = 3.79   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: kxSce-ipaV5GwiN8Ntq2rg r_ui = 4.00   est = 3.85   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 6D5naX_EO6t4L2IGuuSSvA r_ui = 4.00   est = 2.84   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 6_NxWHy4flcwAlxMlK3ONQ r_ui = 4.00   est = 3.88   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: L36IfJdif0rvLfTkQr9hOg r_ui = 4.00   est = 3.62   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: bMcozrCMkppinCDu0C5ueA r_ui = 4.00   est = 3.96   {'was_impossible': False}
u

## Accuracy Metrics
### Root Mean Square Error (RMSE)

> Calculate the RMSE between the predicted rating and the acutal rating given by the sample user

In [34]:
accuracy.rmse(pred_list)

RMSE: 0.7857


0.7856604148733504

> Classify reviews with 3 or more stars as Recommended/Positive and reviews with less than 3 stars as Not Recommended/Negative <br>
Calculate the Precision, Recall, and f1 scores

### Precision, Recall, and F1 Score

In [15]:
def precision_recall_f1_at_k(predictions, threshold=3):
    rating_list = []
    for n in range(len(pred_list)):
        rating_list.append([pred_list[n][2],pred_list[n][3]])
    n_rel = sum((true_r[0] >= threshold) for true_r in rating_list)
    n_rec_k = sum((est[1] >= threshold) for est in rating_list)
    n_rel_and_rec_k = sum(((rating[0] >= threshold) and (rating[1] >= threshold)) for rating in rating_list)
    precision = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
    recall = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
    f1 = 2*(precision*recall)/(precision+recall)

    return precision, recall, f1

> Precision: Ability of the algorithm to not recommend a restaurant that the user has given a negative review for <br>
Recall: Ability of the algorithm to recommend a restaurant that the user has given a positive review for <br>
f1 score: Weighted mean of the precision and recall metrics

In [16]:
user_precision, user_recall, user_f1 = precision_recall_f1_at_k(pred_list)
print('Precision Score: {0:0.2f}'.format(user_precision*100))
print('Recall Score: {0:0.2f}'.format(user_recall*100))
print('f1 Score: {0:0.2f}'.format(user_f1*100))

Precision Score: 93.92
Recall Score: 96.33
f1 Score: 95.11


### Predict Sample User's Restaurant Review Scores
Predictions across all the restaurants in the dataset (even those not visited by the user)

In [35]:
pred_list = list()
for business_id in business_id_dict:
    if (df_new_user['business_id'] == business_id).any():
        r_ui = df_new_user.loc[df_new_user['business_id'] == business_id, 'rating'].iloc[0]
        pred = algo.predict(user_id, business_id, r_ui = r_ui, verbose=True)
    else:
        pred = algo.predict(user_id, business_id, verbose=True)
    pred_list.append(pred)

user: ggl6fl-PM5O1WrdReL0l4A item: jGennaZUr2MsJyRhijNBfA r_ui = None   est = 3.59   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: buTr0hMiyDbjQvFJgm-Eew r_ui = None   est = 3.75   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 3ME_CSB1bo4F0QMhQRUeOA r_ui = None   est = 3.73   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: XDv29FffNd2dWnDOtZP-wg r_ui = None   est = 3.65   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: _p53vrNvW47kNZvRzTQL8A r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: L-Stk9lkVfPSFQlrFukCwA r_ui = None   est = 3.52   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: V0rGM46G0x2cy9RfUt1qPQ r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: fD9EopsUWvimsi4eaS7uMA r_ui = None   est = 3.88   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: SHHxhkzbKBKvv2Mqe2Ylrw r_ui = None   est = 3.68   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: A7DLDlOilcKBD4xFY24QJA r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: xG8nb41yshMoYYSwe_JvMw r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 5gJ54ZorEYdQK7TAD7Yz7g r_ui = None   est = 3.86   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: gz8aXeMOCSjJ4k1WOjgg-g r_ui = None   est = 2.78   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: L3Vp0LhPn1DMKtGillUvgQ r_ui = None   est = 3.56   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: buF9druCkbuXLX526sGELQ r_ui = None   est = 3.49   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: XeGoe2eGu9K4qpL-MWGGPA r_ui = None   est = 4.25   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: l7_-Ce9L_gAi5GDGgGzxZQ r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: EwFdA1bvaiUi_WQqdFeIiw r_ui = None   est = 4.46   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: yXVgVKV-Ek_tE9EZXMVTEQ r_ui = None   est = 3.05   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: QHxG1x_AtwKayPYnehzf2w r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: Pq-JHKwzS6reV-a56AmDyQ r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: TjjbmJCiJULe9XWzisxjFQ r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: U1xRbuGPwn7xAW9-b_CiEQ r_ui = 3.00   est = 3.38   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: SeCCwykRJ7Y7S5Qok4px2w r_ui = None   est = 3.55   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: i6tYcwgNzFN76rd6aUO37g r_ui = None   est = 4.10   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: EA10YCkaxzRoCEOB38xv1Q r_ui = None   est = 3.81   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: PAba4AUZoodeim8AOPCDqQ r_ui = None   est = 3.68   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: NTduNrzztKAcZtg9wEET9w r_ui = None   est = 3.62   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 9tEBtu2x3LZFB5eSJebh7g r_ui = None   est = 3.92   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: PTGn0NTV-GKtwSV80xortA r_ui = None   est = 4.00   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: lCMN0raxlHnBFDoLgkz4sw r_ui = None   est = 4.16   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: tofcAQf-ZGX9wtuAm6llJQ r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 7WFFKd32_9UxZ7ShTQbCQg r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: zmZ3HkVCeZPBefJJxzdJ7A r_ui = None   est = 3.49   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: BCFDatZ72MzHNqcJI4yy-w r_ui = None   est = 2.43   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: bOkLeien1ra8x-7R9E8iYQ r_ui = None   est = 3.31   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: xdpH27x6qGSG21LLa6TaXQ r_ui = None   est = 3.85   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: eXmwOusJjdj-n9vy0wlgLw r_ui = None   est = 2.80   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: q9s0qz4VdwwLhcRiMIyZjw r_ui = None   est = 4.24   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 5HMXgD_gui5n0Tc_hadesg r_ui = 4.00   est = 3.81   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: uMRH_0ClIFshQmkPpoYLrg r_ui = None   est = 3.33   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: Rp976b1NTI_U-ONHKDA8yg r_ui = None   est = 3.17   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: yx_tOt-p-pxWNWAc2ebTcA r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: hdH1zkYvK8YwqDGZEUdAfQ r_ui = 3.00   est = 3.87   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 0QBpqLaKqVKlHTSsrqBHdQ r_ui = None   est = 2.89   {'was_impossible': False}
u

...

user: ggl6fl-PM5O1WrdReL0l4A item: efKrQOMu0gLiTQ2uG0NX_A r_ui = 4.00   est = 3.77   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: rDbOUnFEaY1jvkW4W5cv5A r_ui = None   est = 3.07   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: gl151ZpwS6q1jpaEdZnVTg r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: XXX3oetPpn-17VV_8i6u5g r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: nu5lbIOitciGPD4B_56PyQ r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 4NocTXHleji0hihMhMPnJA r_ui = None   est = 3.69   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: WC4p6QkUVuAt9gUmxGkV3Q r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: iTi0fcU_dUDn6RET3Bn_aQ r_ui = None   est = 3.37   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: BpyMTC_c8csEFjKnatRvlQ r_ui = None   est = 3.55   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: p-RYIhT5AcYiRNOsZBIkkg r_ui = None   est = 3.32   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: qRB9s_VG_whQMOXZPZafnw r_ui = None   est = 2.98   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: RcnPteK5TdK41In_KGU0BQ r_ui = None   est = 4.33   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: aOISYaVK7_EoScjBhJ1NZQ r_ui = None   est = 4.47   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: FOnFjJRuCjwD5sGN9-1rCA r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: e98R4S7OgciVFvLlM43d1g r_ui = None   est = 3.54   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: cg8mfYBkdhaPrvEVblm7gQ r_ui = None   est = 4.23   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: xk9iS8CqpsXpfuiFmT5omw r_ui = None   est = 4.05   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: KNGPr-Gpues7WhZXAtlvmA r_ui = None   est = 4.12   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: TM2s1Pxb1z6y4Xd7kEEaWA r_ui = None   est = 3.59   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: bwNwEWulr97oCJ6ux9VVoA r_ui = None   est = 3.90   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 7R6MJALUagO2pX1sceR6yQ r_ui = None   est = 3.57   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: S27Qg-YHLOshg__tJbijuw r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: YGSacISAJEsEGrBKFwR0Gg r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: -I4bxb04YkCSNCbFWto7Gw r_ui = 4.00   est = 4.02   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: kM9FKlmLgVkXec580gq7Dg r_ui = None   est = 4.07   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: XEdmeFVidV1YjdTgagdCUg r_ui = None   est = 3.46   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: n2sIR2-c0qG2tN7s07vnQw r_ui = None   est = 3.69   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: Bctfmv69UjEk6bcL_PkoDg r_ui = None   est = 3.22   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 3sHtQuytlkktze3ETt1BGg r_ui = None   est = 3.12   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: GBnUoBFDPIHYXyhLMDQRzQ r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: NslwiBEmTw7qXeMNAZTkUQ r_ui = None   est = 3.88   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: da1yiInoYkLmJCJvlhggIQ r_ui = None   est = 3.44   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: nwU1tKh-D68B7Z44DbDwXg r_ui = None   est = 3.83   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: tyuzgXFQGEkM5A9e6HmSLA r_ui = 2.00   est = 3.09   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: OKQuK8TcPQC5Kzo-O-OYDw r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: mpBS1SGGUJN3SZ24axK0iQ r_ui = None   est = 3.68   {'was_impossible': False}
u

user: ggl6fl-PM5O1WrdReL0l4A item: 757zI3bNk4CZ9cql3T6rbQ r_ui = None   est = 4.20   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: KbPL1DxBo2fGMTEkU2yoTg r_ui = None   est = 3.76   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: SSmHkP0tZ4TKn_YUOEkVUg r_ui = None   est = 3.89   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: EMgU4Dz69B4A2_GgNlzqaw r_ui = None   est = 3.97   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: B3SYmXQ7Xtobk_XWJIdAnQ r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: XMxgZsQnBZ6r02QR2kIcWQ r_ui = None   est = 3.54   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: qS_sydoMwEMuBU0ByOM-JA r_ui = None   est = 3.79   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: 0_Gasj4t2pH5mNvYyBDPaQ r_ui = None   est = 3.68   {'was_impossible': False}
user: ggl6fl-PM5O1WrdReL0l4A item: _0d408AYQPBJ0ywAaFBF3g r_ui = None   est = 3.43   {'was_impossible': False}
u

## Recommender Algorithm Set up
> Sort the predicted review scores from high to low <br>
Grab the top 25 review scores and print out the restaurants they belong to

In [42]:
def find_topn(predictions, n, user_id):
    topn = defaultdict(list)
    for uid, bid, _, est, _ in predictions:
        topn[uid].append((bid, est))

    for uid, est in topn.items():
        est.sort(key=lambda x: x[1], reverse=True)
        topn[uid] = est[:n]

    user_results = pd.DataFrame.from_dict(topn).transpose().loc[user_id]
    top_restaurant_id = user_results.loc[0][0]
    
    print('Top {0} recommendations for user {1}:\n'.format(n, get_user_attr(user_id, 'name')))
    print('   Restaurant:                                  Predicted Rating:              Restaurant ID:\n')
    for i in range(n):
        print('{:<3d}{:<50s}{:1.2f}{:>40s}'.format(i+1, get_business_attr(user_results.loc[i][0], 'name'), user_results.loc[i][1], user_results.loc[i][0]))
        
    return user_results, top_restaurant_id

## Final Recommendations for Sample User

### Top 25 Recommendations for Sample User

In [37]:
user_topn, top_restaurant_id = find_topn(pred_list, 25, user_id)

Top 25 recommendations for user Michael:

   Restaurant:                                  Predicted Rating:              Restaurant ID:

1  Harry's Cheese and Cold Cuts                      4.90                  w7JR1y-TCXmWdFKDLW83eQ
2  Recreo Coffee & Roasterie                         4.77                  ZjNw6MQ9EyS16COI465xbQ
3  Jolie Tea Company                                 4.75                  Lcl3E29jwhK3HUYzvoA_1A
4  Tutto Italiano                                    4.72                  che285DuI-eo98PriDIerg
5  MA-France                                         4.71                  jdNyDx7IuAruyUju091z9Q
6  Ciao! Pizza & Pasta                               4.68                  yQL8SrSETbbCI1U5esVJQw
7  Fred's Franks                                     4.67                  H2KAvHOGz1wRHJoS-u4fpQ
8  O Ya                                              4.63                  87f7kR7nTz8WHnmtLM_S6w
9  Tony's Market                                     4.63                  9Ye0

## Top Recommended Restaurant
### General business information

In [38]:
print_business_info(top_restaurant_id)

business_id: w7JR1y-TCXmWdFKDLW83eQ
name: Harry's Cheese and Cold Cuts
address: 98 Blackstone St
city: Boston
state: MA
postal_code: 02109
latitude: 42.3615179
longitude: -71.0564517
stars: 5.0
review_count: 57
is_open: 1
attributes: {'RestaurantsTakeOut': 'True', 'BusinessAcceptsCreditCards': 'True', 'GoodForKids': 'True', 'OutdoorSeating': 'False', 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': True, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}", 'RestaurantsGoodForGroups': 'False', 'RestaurantsAttire': "'casual'", 'RestaurantsReservations': 'False', 'BusinessParking': "{'garage': False, 'street': True, 'validated': True, 'lot': False, 'valet': False}", 'NoiseLevel': "u'average'", 'Caters': 'True', 'WiFi': "u'no'", 'RestaurantsDelivery': 'False', 'RestaurantsPriceRange2': '1', 'BikeParking': 'False', 'HasTV': 'True'}
categories: Cheese Shops, Food, Meat Shops, Delis, Restaurants, Specialt

### Restaurant Reviews

In [39]:
get_business_reviews(top_restaurant_id)

Name:                 User ID:               User Total Review Count         Rating
Tanya          ILZtzt-3oEmLu8c1rfeJuw                 201                      5.0
Steven         0Zq95YruU7IvCsvzFOSSzA                 170                      4.0
Jacquelyn      F0_FBGWlJS7z_qTDm9dJSA                 281                      5.0
boom           bje1P4XGW9lv79iuSKBgiw                  98                      5.0
Lauretta       W_EQFi2fXpBtuRm510cCpA                 474                      5.0
Vivian         nl8gWLDo6U6MjqzbBmE_9A                1871                      5.0
Sarah          5LT0UfheP3c-_xKqmZDgxQ                  51                      4.0
Dorothy        W-SyRg4KEOwcRRA_hj3Txw                 467                      5.0
Max            p2NLkCmcPp1-Mr8x9TXXcQ                  73                      5.0
Chewie         -GnSkFtT3axpOdkSxmx7Zw                  49                      5.0
Benjamin       4v30Vm27ViLTv4pRQNCIPA                  29                      5.0
Ste

## Sample User Profile

In [40]:
print_user_info(user_id)

user_id: ggl6fl-PM5O1WrdReL0l4A
name: Michael
review_count: 1434
yelping_since: 2008-01-28 07:34:15
useful: 3376
funny: 1329
cool: 1658
elite: 2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,20,20
friends: [list of user ids]
fans: 120
average_stars: 3.85
compliment_hot: 51
compliment_more: 12
compliment_profile: 9
compliment_cute: 1
compliment_list: 1
compliment_note: 49
compliment_plain: 122
compliment_cool: 145
compliment_funny: 145
compliment_writer: 55
compliment_photos: 32
